In [22]:
import sklearn
import pandas as pd
import numpy as np
import json
from pprint import pprint

In [24]:
!pwd

/home/ubuntu/repos/uber_app


In [25]:
raw_data = pd.read_json("./uber_data_challenge v2.json")

In [38]:
raw_data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,uber_black_user,weekday_pct
0,3.67,5,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5,5,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [37]:
raw_data['last_trip_date'] = pd.to_datetime(raw_data['last_trip_date'])
raw_data['signup_date'] = pd.to_datetime(raw_data['signup_date'])

In [ ]:
current_date = max(raw_data['last_trip_date'])

In [49]:
type(current_date)

pandas.tslib.Timestamp

In [ ]:
diffs = current_date - raw_data['last_trip_date']

In [52]:
type(diffs[0])

pandas.tslib.Timedelta

In [53]:
raw_data['days_since_last_trip'] = current_date - raw_data['last_trip_date']

In [54]:
raw_data['active'] = raw_data['days_since_last_trip'].days < 30

AttributeError: 'Series' object has no attribute 'days'